### 寫成函式

In [ ]:
#colab環境設定
!pip3 install pyclustering
!pip install apyori

from google.colab import drive
from pyclustering.cluster.kmeans import kmeans, kmeans_visualizer
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.samples.definitions import FCPS_SAMPLES
from pyclustering.utils.metric import distance_metric, type_metric
from pyclustering.utils import read_sample
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from scipy.stats import skew,norm, kurtosis
from scipy.spatial.distance import cdist
from time import time
from mlxtend.preprocessing import TransactionEncoder
from apyori import apriori
import pandas as pd
import numpy as np
import numpy
import statistics
import csv
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def Category_one(file_name):
	data = pd.read_csv(file_name, index_col=0)
	df = pd.DataFrame(data)
	features_arr = np.empty(shape=4,dtype=float)
	Num_instances, Num_features = df.shape#Number of instances,Number of features
	#for x in range(int(Num_instances/50),Num_instances):#減少資料量
	#	df=df.drop(x, axis = 0)
	#print(df.dtypes)
	Data_dim=Num_instances/Num_features#Data dimensionality
	#print("Num_instances=",Num_instances,"Num_features=",Num_features,"Data_dim=",Data_dim)
	features_arr[0]=Num_instances
	features_arr[1]=Num_features
	features_arr[2]=Data_dim
	entropy_arr = np.zeros(shape=(Num_features-1))
	i=0;
	for col in df.drop(columns='class'):
		probability = df[col].value_counts(normalize=True)
		entropy=-1*np.sum(np.log2(probability)*probability)
		entropy_arr[i]=entropy
		i=i+1
	mean_entropy=entropy_arr.mean()
	#print("mean_entropy",mean_entropy)#mean entropy of all attributes
	features_arr[3]=mean_entropy

	return features_arr

def isInList(aList, item):
    if item in aList:
        return 1
    else:
        return -1

# Function to remove strings (e.g., white space) from the list items


def removeEmptyValues(aList, symbolToRemove):
    while (True):
        val = isInList(aList, symbolToRemove)
        if (val == 1):
            aList.remove(symbolToRemove)
        else:
            break

# Function to create a list of lists from a CSV file containing transaction data


def deletecol(data, i):
    for row in data:
        del row[i]
    return data

# Function to create a list of lists from a CSV file containing transaction data


def generateListOfListsFromCSV(fullFileName):

    with open(fullFileName, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f, delimiter=",")
        data = list(reader)

    listOfLists = []

    for i in range(len(data)):
        removeEmptyValues(data[i], '')
        listOfLists.append(data[i])

    return listOfLists

# Function to create a one-hot encoded DataFrame object from the Python list of lists


def oneHotEncodedDataFrame(data):

    # Transform the transaction data into a one-hot encoded NumPy boolean array
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)

    # Read the one-hot encoded data as a Pandas DataFrame object
    dataFrame = pd.DataFrame(te_ary, columns=te.columns_)

    return dataFrame

# Function to create a list of items & baskets from a one-hot encoded DataFrame object


def generateItemsBaskets(dataFrame):
    # Create a list of items from the DataFrame object
    items = []
    for col in dataFrame.columns:
        items.append(col)

    # Create a list of baskets from the DataFrame object
    baskets = []
    transaction_id = 1

    for i in range(len(dataFrame)):
        itemset = []
        for j in range(len(items)):
            if (dataFrame.iloc[i][j] == True):
                itemset.append(items[j])
        baskets.append({transaction_id: itemset})
        transaction_id += 1

    return items, baskets


def bruteForceFrequentItemsets(dataFrame, min_support=2, max_len=None):

    # Create a list of items & baskets from a one-hot encoded DataFrame object
    items, baskets = generateItemsBaskets(dataFrame)

    # Declare a list to store frequent 2-itemsets
    freq_itemsets = []

    for i in range(len(items)):

        for j in range(len(items)):

            if (j != i and j > i):
                basket_count = 0
                basket_item = 1
                for b in baskets:
                    val1 = isInList(b.get(basket_item), items[i])
                    val2 = isInList(b.get(basket_item), items[j])
                    if (val1 != -1 and val2 != -1):
                        basket_count += 1
                    basket_item += 1

                if (basket_count >= min_support):
                    freq_itemsets.append([items[i], items[j]])

    return freq_itemsets

def Category_two(file_name):
	itemSetList = generateListOfListsFromCSV(file_name)
	# print(itemSetList)
	itemSetList.pop(0)
	#print("=======================================================================================================================================")

	itemSetList = deletecol(itemSetList, (len(itemSetList[0])-1))    #若CSV中有column要刪除就用這個,刪除column class
	#print(oneHotEncodedDataFrame(itemSetList))
	# print(freqItemSet)

	association_rules = apriori(
		itemSetList, min_support=0.05, min_confidence=0, min_lift=1, min_length=1)
	association_results = list(association_rules)
	# items = [x for x in pair]
	# print("Support: " + str(item[1]))
	features_arr = []
	#print("1-itemset :")
	one_itemset_arr = []
	for i in range(0, len(association_results)):
		if (len(association_results[i][0]) == 1):
			#print(association_results[i][0],
			#	"Support : ", association_results[i][1])
			one_itemset_arr.append(association_results[i][1])
	one_itemset_arr = np.array(one_itemset_arr)
	one_itemset_arr_sort = np.sort(one_itemset_arr)#sorted in ascending order according
	one_itemset_arr_size=len(one_itemset_arr_sort)
	features_arr.append(one_itemset_arr_sort[0])
	features_arr.append(one_itemset_arr_sort[int((one_itemset_arr_size/8))])
	features_arr.append(one_itemset_arr_sort[int((one_itemset_arr_size/8*2))])
	features_arr.append(one_itemset_arr_sort[int((one_itemset_arr_size/8*3))])
	features_arr.append(one_itemset_arr_sort[int((one_itemset_arr_size/8*4))])
	features_arr.append(one_itemset_arr_sort[int((one_itemset_arr_size/8*5))])
	features_arr.append(one_itemset_arr_sort[int((one_itemset_arr_size/8*6))])
	features_arr.append(one_itemset_arr_sort[int((one_itemset_arr_size/8*7))])
	features_arr.append(one_itemset_arr_sort[one_itemset_arr_size-1])

	two_itemset_arr = []
	#print("2-itemset :")
	for i in range(0, len(association_results)):
		if (len(association_results[i][0]) == 2):
			#print(association_results[i][0],
			#	"Support : ", association_results[i][1])
			two_itemset_arr.append(association_results[i][1])
	two_itemset_arr = np.array(two_itemset_arr)
	two_itemset_arr_sort = np.sort(two_itemset_arr)#sorted in ascending order according
	two_itemset_arr_size=len(two_itemset_arr_sort)
	features_arr.append(two_itemset_arr_sort[0])
	features_arr.append(two_itemset_arr_sort[int((two_itemset_arr_size/8))])
	features_arr.append(two_itemset_arr_sort[int((two_itemset_arr_size/8*2))])
	features_arr.append(two_itemset_arr_sort[int((two_itemset_arr_size/8*3))])
	features_arr.append(two_itemset_arr_sort[int((two_itemset_arr_size/8*4))])
	features_arr.append(two_itemset_arr_sort[int((two_itemset_arr_size/8*5))])
	features_arr.append(two_itemset_arr_sort[int((two_itemset_arr_size/8*6))])
	features_arr.append(two_itemset_arr_sort[int((two_itemset_arr_size/8*7))])
	features_arr.append(two_itemset_arr_sort[two_itemset_arr_size-1])
	#print("features_arr=",features_arr)

	return features_arr


#Function to  create % of values of S1 in the interval [0,0.1], . . . , (0.9,1.0]

def create_S1_fre_arr(S):
	min_S=min(S)#minimum distance values in S
	max_S=max(S)#maximum distance values in S
	max_min_diff_distance=max_S-min_S
	S_size=len(S)
	S_1 = np.zeros(shape=S_size)
	S_1_fre = np.zeros(shape=10)#The frequencies of distance values in each interval of S
	for i in range(len(S)-1):
		S_1[i]=(S[i]-min_S)/max_min_diff_distance
		if S_1[i]<0.01:
			S_1_fre[0]=S_1_fre[0]+1
		elif S_1[i]<0.02:
			S_1_fre[1]=S_1_fre[1]+1
		elif S_1[i]<0.03:
			S_1_fre[2]=S_1_fre[2]+1
		elif S_1[i]<0.04:
			S_1_fre[3]=S_1_fre[3]+1
		elif S_1[i]<0.05:
			S_1_fre[4]=S_1_fre[4]+1
		elif S_1[i]<0.06:
			S_1_fre[5]=S_1_fre[5]+1
		elif S_1[i]<0.07:
			S_1_fre[6]=S_1_fre[6]+1
		elif S_1[i]<0.08:
			S_1_fre[7]=S_1_fre[7]+1
		elif S_1[i]<0.09:
			S_1_fre[8]=S_1_fre[8]+1
		else :
			S_1_fre[9]=S_1_fre[9]+1
	
	S_1_fre=S_1_fre/S_size
	#print("S_1_fre=",S_1_fre)#debug
	return S_1_fre

#Function to  create % of values of S2 in the interval [0,1), . . . , [3,+∞)

def create_S2_fre_arr(S,Mean_S,Standard_deviation_S):
	S_2 = np.zeros(shape=len(S))#Each distance value s ∈ S is normalized
	S_2_fre = np.zeros(shape=4)#The frequencies of distance values in each interval are computed
	S_size=len(S)
	for i in range(S_size-1):
		S_2[i]=(S[i]-Mean_S)/Standard_deviation_S
		if S_2[i]<1:
			S_2_fre[0]=S_2_fre[0]+1
		elif S_2[i]<2:
			S_2_fre[1]=S_2_fre[1]+1
		elif S_2[i]<3:
			S_2_fre[2]=S_2_fre[2]+1
		else :
			S_2_fre[3]=S_2_fre[3]+1
	S_2_fre=S_2_fre/S_size
	#print("S_2_fre=",S_2_fre)#debug
	return S_2_fre

def Category_three(file_name,Num_instances):
	dataset = generateListOfListsFromCSV(file_name)
	dataset_onehotencoded = oneHotEncodedDataFrame(dataset)
	dataset_onehotencoded = dataset_onehotencoded.astype(int)
	Num_instances_int=Num_instances.astype(int)
	S_size=(Num_instances_int-1)*Num_instances_int/2
	S = np.zeros(shape=int(S_size))#The Euclidean distance values between all pairs of the data objects in dataset
	i=0
	for x in range(1,Num_instances_int):
		for y in range(x+1,Num_instances_int):
			S[i] = numpy.linalg.norm(dataset_onehotencoded.loc[x]-dataset_onehotencoded.loc[y])
			i=i+1
	features_arr = np.empty(5,dtype=float)
	Mean_S=S.mean()#Mean of S
	#print("Mean_S=",Mean_S)#debug
	features_arr[0]=Mean_S
	Variance_S=statistics.variance(S)#Variance of S
	#print("Variance_S=",Variance_S)#debug
	features_arr[1]=Variance_S
	Standard_deviation_S = numpy.std(S)#Standard deviation of S
	#print("Standard_deviation_S=",Standard_deviation_S)#debug
	features_arr[2]=Standard_deviation_S
	Skew_S=skew(S)#Skewness of S
	#print("Skew_S=",Skew_S)#debug
	features_arr[3]=Skew_S
	Kurtosis_S=kurtosis(S)#Kurtosis of S
	#print("Kurtosis_S=",Kurtosis_S)#debug
	features_arr[4]=Kurtosis_S
	S1_fre_arr=create_S1_fre_arr(S)
	features_arr=np.hstack([features_arr,S1_fre_arr])
	S2_fre_arr=create_S2_fre_arr(S,Mean_S,Standard_deviation_S)
	features_arr=np.hstack([features_arr,S2_fre_arr])

	return features_arr


def metaFeatureCollection(file_name):

	features_arr=np.empty(4,dtype=float);
	Sta_info_arr = Category_one(file_name)#Statistical and information theory-based
	#print("Sta_info_arr=",Sta_info_arr)
	features_arr = Sta_info_arr
	#Category_two(file_name)
	Struc_info_arr=Category_two(file_name)#Structural information-based
	features_arr=np.concatenate([features_arr,Struc_info_arr])
	Dis_info_arr = Category_three(file_name,features_arr[0])#Distance information-based
	#print("Dis_info_arr=",Dis_info_arr)
	features_arr=np.concatenate([features_arr,Dis_info_arr])
	#print("features_arr=",features_arr)

	return features_arr

In [ ]:
def euclidean_dis(x, y):
    return np.sqrt(np.sum((x - y) ** 2))
def manhattan_dis(x, y):
    return np.sum(np.abs(x - y))
def StandardizedEuclidean(x, y):
    x = np.array(x)
    y = np.array(y)

    X = np.vstack([x, y])
    sigma = np.var(X, axis=0, ddof=1)
    # print(sigma)
    return np.sqrt(((x - y) ** 2 / sigma).sum())
def cosin_dis(x, y):
    x = np.array(x)
    y = np.array(y)
    return 1 - np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
    
# metrix = distance_metrix(type_metrix.USER_DEFINED, func=euclidean_dis)

def get_dismetrix(dis_method):
    if dis_method == 'euclidean':
        metric = distance_metric(type_metric.EUCLIDEAN)
    elif dis_method == 'manhattan':
        metric = distance_metric(type_metric.MANHATTAN)
    elif dis_method == 'chebyshev':
        metric = distance_metric(type_metric.CHEBYSHEV)
    elif dis_method == 'minkowski':
        metric = distance_metric(type_metric.MINKOWSKI)
    elif dis_method == 'canberra':
        metric = distance_metric(type_metric.CANBERRA)
    elif dis_method == 'stand_eulidean':
        metric = distance_metric(type_metric.USER_DEFINED, func=StandardizedEuclidean)
    elif dis_method == 'cosin_dis':
        metric = distance_metric(type_metric.USER_DEFINED, func=cosin_dis)
    return metric

def get_cluster_object(Data, initial_centers, metric, algo):
    if algo == 'kmeans':
        kmeans_instance = kmeans(Data, initial_centers, metric=metric)
        return kmeans_instance
def BuildCluster(Data, dis_method, algo, n_clusters):

    initial_centers = kmeans_plusplus_initializer(Data, n_clusters).initialize()
    initial_centers = np.array(initial_centers)

    # Create instance of K-Means algorithm with prepared centers.
    metric = get_dismetrix(dis_method)
    kmeans_instance  = get_cluster_object(Data, initial_centers, metric, algo)
    # kmeans_instance = kmeans(Data, initial_centers, metric=metric)
    # Run cluster analysis and obtain results.
    kmeans_instance.process()
    clusters = kmeans_instance.get_clusters()
    return clusters
def evaluate(clusters, cli, E):
    score = 0
    if E == 'f-score':
        y_pred = [0 for i in range(len(cli))]
        for j in range(len(clusters)):
            for i in range(len(clusters[j])):
                y_pred[clusters[j][i]] = cli[clusters[j]].value_counts().index[0]
        score = f1_score(cli, y_pred, average='macro')
    return score
def Meta_target_identification(D = [], S = ['euclidean', 'manhattan'],
                   E = 'f-score', C = 'kmeans', F = 'single'):
    Evalmetric = []
    DSRank = []
    Y = []
    for i in range(len(D)):
        cli = D[i]['class']
        D[i] = D[i].drop(['class'], axis=1) 
        n_clusters = len(cli.value_counts())
        for j in range(len(S)):
            Clusters = BuildCluster(D[i], S[j], C, n_clusters)
            value = evaluate(Clusters, cli, E)
            Evalmetric.append(value)
        # dsrank is sorting evalmetric descending return index
        DSRank = np.argsort(Evalmetric)[::-1]
        #print(DSRank)
        if F == 'single':
            optimal_index = DSRank[0]
            optimal_dis_measure = S[optimal_index]
            Y.append([optimal_dis_measure])
        else:
            Y.append(DSRank)
        Evalmetric.clear()
    return Y

In [ ]:
def EuclideanDist (Xnew, neighbor) :
  temp = 0
  for i in range(len(Xnew[0])):
    temp += (Xnew[0][i] - neighbor[i])**2
  return (temp**(1/2))

def NearNeighbors(M, Xnew, k, q):
  neigh = NearestNeighbors(n_neighbors=k)
  neigh.fit(M[0])
  Neighbors = neigh.kneighbors(Xnew)

  #算出每個neighbor到Xnew的距離，並將其倒數設為權重
  weight = [0] * k
  for i in range(k) : 
    #Neighbors[1][0][i]為第i個最接近Xnew的neighbor index
    #neighbor為由第i個最接近Xnew的neighbor中的Training dataset，所萃取出來的Data
    neighbor = M[0][Neighbors[1][0][i]]
    d = EuclideanDist(Xnew, neighbor)
    if d!=0 :
      weight[i] = 1/d
    else : 
      #將無限大改為1
      weight[i] = 1

  #算出每個Measure對應於其他neighbors的權重值
  scores = [0] * q
  for i in range(q):
    for j in range(k):
      #Neighbors[1][0][j]為距離Xnew第j近的neighbor index
      #y為此index所對應到的measures list
      y = M[1][Neighbors[1][0][j]]
      #找出第i個Measure在此list中的index，以此作為其重要性
      ind = y.index(i)
      scores[i] += weight[j] * ( q - ind )
  return np.argsort(scores)


In [ ]:
#training DataSet that was inputted have to name meta-target as "class"
#data_path need to specified in advance.
# encode feature

S = ['euclidean', 'manhattan', 'chebyshev','canberra', 'stand_eulidean', 'cosin_dis']# , 'manhattan'
E = ['f-score']
C = ['kmeans']
F = ['ranking'] # 'single', 
data_path = ['/content/drive/MyDrive/Colab_Notebooks/lowbwt.csv','/content/drive/MyDrive/Colab_Notebooks/dataset_13_breast-cancer.csv',
                '/content/drive/MyDrive/Colab_Notebooks/dataset_10_lymph.csv','/content/drive/MyDrive/Colab_Notebooks/cholesterol.csv',
             '/content/drive/MyDrive/Colab_Notebooks/primary-tumor.csv','/content/drive/MyDrive/Colab_Notebooks/mushrooms.csv',
             '/content/drive/MyDrive/Colab_Notebooks/DowJones.csv','/content/drive/MyDrive/Colab_Notebooks/diabetes.csv',
             '/content/drive/MyDrive/Colab_Notebooks/car_evaluation.csv']

Database = []
for i in range(len(data_path)):
  Data = pd.read_csv(data_path[i])
  for j in range(len(Data.columns)):
    Data[Data.columns[j]]= Data[Data.columns[j]].astype('category')
    Data[Data.columns[j]] = Data[Data.columns[j]].cat.codes
  Database.append(Data)

Y = Meta_target_identification(D = Database, S = S, E = E[0], C = C[0], F = F[0])
X = []
M = [X, Y]
q = len(S)
# 自定義參數K，代表欲選擇Neighbors的數量
k = 2
#挑選一個不再Training data中的dataset
Xnew = metaFeatureCollection('/content/drive/MyDrive/Colab_Notebooks/heart2.csv')

#將Database的每個dataset extracte後加到X中
for i in range(len(data_path)):
  X.append(metaFeatureCollection(data_path[i]))
#將X的參數型態改成list
for i in range(len(M[0])) : 
  M[0][i] = M[0][i].tolist()
#將Y的參數型態改成list
for i in range(len(M[1])) : 
  M[1][i] = M[1][i].tolist()

#由於Kneighbor不支援多個target value，因此先將Y做等級分群
Y_component = []
for j in range(len(M[1][i])) :
  temp = []
  for i in range(len(M[1])) :
    temp.append(M[1][i][j])
  Y_component.append(temp)

ynew = NearNeighbors(M, [Xnew], k, q)
print(ynew)

[0 1 2 3 5 4]
